# 1. Importar tablas

In [1]:
/ 1.1. Importar tabla nombres
names:("*J";enlist ";") 0:`$":data/names.csv";

update sums frecuencia from `names;
/show names;

In [2]:
/ 1.2. Importar tabla apellidos
surnames:("*JJ";enlist ";") 0:`$":data/surnames.csv";

update sums frec1, sums frec2 from `surnames;
/show surnames;

In [3]:
/ 1.3. Importar tabla lista cnae
cnae:("**";enlist ";") 0:`$":data/cnae.csv";

update "S"$codigo from `cnae;
/show cnae;

In [4]:
/ 1.4. Importar tabla costes salariales
eacl:("*JFF";enlist ";") 0:`$":data/eacl.csv";

update "S"$division from `eacl;
eacl:  update anio: {(2025 2024),x}'[anio],
              salario: {((1.05*1.0441;1.05)*(2#first x)),x}'[salario],
              resto_costes: {((1.05*1.0441;1.05)*(2#first x)),x}'[resto_costes]
       from `division xgroup eacl;
/show eacl

# 2. Variables globales

In [5]:
n_Employees: 1000;                           /(n)umero de empleados

t_names: last names[`frecuencia];            /(t)otal de nombres
t_surn1: last surnames[`frec1];              /(t)otal de primeros apellidos
t_surn2: last surnames[`frec2];              /(t)otal de segundos apellidos

l_cnae: cnae[`codigo];                       /(l)ista cnae

i_date: 2008.01.01                           /(i)nicio fecha


l_division: ("I"$ string distinct key eacl)  /(l)ista (division) cnae

# 3. Funciones

## 3.1. employees

In [6]:
gnameaux:{[tabla;colValor; colFreq; total]                               /(g)en (name)-(aux)iliar
    tabla[colValor] prev[tabla[colFreq]] bin rand total};

/---------- funciones de cada campo de empleado ----------

gnif:{`${x,"TRWAGMYFPDXBNJZSQVHLCKE"[("J"$x) mod 23]}8 ? .Q.n};          /(g)en (nif)

gname: {" " sv gnameaux'[                                                /(g)en (name)
    (`names,       `surnames,  `surnames);
    (`nombre,      `apellido,  `apellido); 
    (`frecuencia,  `frec1,     `frec2);
    (t_names,      t_surn1,    t_surn2)]}

gnaf:{{x,-2#"0",string ("J"$x) mod 97}"28", 8 ? .Q.n};                   /(g)en (naf)

gconty:{`fijodis`indef`temp  0 0.15 0.40 bin rand 1f};                   /(g)en (con)tract (ty)pe

gworks:{`parcial`completo  0 0.35 bin rand 1f};                          /(g)en (work) (s)chedule

gcongr:{1+rand 11};                                                      /(g)en (con)tribution (gr)oup

gcnae:{rand l_cnae};                                                     /(g)en (cnae)

/---------- funciones de cada campo de empleado ----------

gemployee:{`nif`name`naf`contractType`workingSchedule`contrGroup`cnae!   /(g)en (employee) 
           ((@[;::]')(gnif;gname;gnaf;gconty;gworks;gcongr;gcnae))};

cemployee:{gemployee each x#null};                                       /(c)reate (employee) 

In [7]:
type ((@[;::]')(gnif;gname;gnaf;gconty;gworks;gcongr;gcnae))

0h


## 3.2. temp_month

In [8]:
gdate:{x+rand .z.d-x};                                                              /(g)en (date)

gdwinmon:{[m;sd;ed]$[m="m"$ed;`dd$ed;`dd$("d"$m+1)-1]-$[m="m"$sd;(`dd$sd)-1i;0i]};   /(g)et (d)ays (w)orked (in) a (mon)th

gmatrd:{[x;m;s;e]gdwinmon'[m[x;];                                                    /(g)et (matr)ix with (d)ays
                        .Q.addmonths[s;x*12];
                        .Q.addmonths[e;x*12]]};

/---------- funciones de cada tipo de contrato ----------

gmindef:{[sd]md:`month$sd;                                                        /(g)en (m)onths (`indef)
             mlist:md+/:til 1+(`month$.z.d)-md-1;
             dlist:gdwinmon[;sd;0Wd]'[mlist];
             flip `month`daysWorked!(mlist;dlist)};

gmtemp:{[sd]md:`month$sd;                                                          /(g)en (m)onths (`temp)
            mlist: md+/:til (`month$gdate(`date$md))-md-1;
            ed:gdate(sd);
            dlist:gdwinmon[;sd;ed]'[mlist];
            flip `month`daysWorked!(mlist;dlist)};

gmfijodis:{[sd]md:`month$sd;                                                      /(g)en (m)onths (`fijodis)
               matrm: (12 cut md+/:til (`month$.z.d)-md-1)[;til 2+rand 10];
               firstl:matrm[0;] where not null matrm[0;];
               ed:(`date$last firstl)+1+rand `dd$(`date$(last firstl)+1)-1; 
               matrd:gmatrd[;matrm;sd;ed]each rank matrm;
               t:flip `month`daysWorked!(raze matrm;raze matrd);
               t where not null t[`month]};

/---------- funciones de cada tipo de contrato ----------

gtabmad:{[ct] sd:gdate[i_date];                                                       /(g)en (tab)le (m)onth (a)nd (d)ate
              funcs: `indef`temp`fijodis!((gmindef; gmtemp; gmfijodis));
              (funcs ct)sd}

## 3.3. temp_remuneration

In [9]:
/---------- funciones de cada tipo de remuneracion ----------

gsb:{[s](`sb;s)};                                                                        /(g)en "(s)alario (b)ase"

ghefm:{[s]if[rand[1.]>.01; :(`;0n)];                                                     /(g)en "(h)oras (e)xtra (f)uerza (m)ayor"
          (`hefm;s*1.75*rand[7]%140)};

gher:{[s]if[rand[1.]>.3; :(`;0n)];                                                       /(g)en "(h)oras (e)xtra (r)esto"
          (`her;s*1.75*(1+rand[7])%140)};

gge:{[s](`ge;s%6)};                                                                      /(g)en "(g)ratificación (e)xtrasalarial"

gse:{[s]if[rand[1.]>.1; :(`;0n)];                                                        /(g)en "(s)alario (e)especie"
          (`se;s*rand .05)};

gpns:{[r]if[rand[1.]>.02; :(`;0n)];                                                      /(g)en "(p)ercepcion (n)o (s)alarial"
          :(`pns;r*.5*rand 1.)};

god:{[r]if[rand[1.]>.02; :(`;0n)];                                                       /(g)en "(o)tras (d)educciones"
          :(`od;r*.5*rand 1.)};

/---------- funciones de cada tipo de remuneracion ----------

gremtu:{[sal;res]{x where not null x} each flip ((@[;sal]')(gsb;ghefm;gher;gge;gse)),        /(g)en (rem)uneration (tu)ples
                                                    ((@[;res]')(gpns;god))}

gtdic:{[cn] eacl `$-2#"0",string first value l_division l_division bin "I"$2#string cn}; /(g)e(t) (dic)tionary

gtabrat:{[cn;mon;wd;ws]dic:gtdic[cn];
                       pos:where dic[`anio] = `year$mon;                               /(g)en (tab)le (r)emuneration (a)nd (t)ype
                       prorat:wd%`dd$(`date$(mon+1))-1;
                       sal:(1%12)*(0.80+rand .1)*prorat*first dic[`salario]pos;
                       res:(1%12)*(0.95+rand .1)*prorat*first dic[`resto_costes]pos;
                       t:`remunerationType`amount!
                       (gremtu[sal;res]);
                       t[`amount]*:$[ws=`parcial;0.5;1];
                       :flip t;};

gtabrat[`8888;2024.12m;30;`completo]

remunerationType amount  
-------------------------
sb               1244.33 
ge               207.3883


## 3.4. remunerations

In [10]:
ctemp_month:{[ct;nf]temp_month: gtabmad[ct];
                    temp_month[`nif]:nf;
                    temp_month};

ctemp_rem:{[mon;dw;nf;ws;cn]temp_rem: gtabrat[cn;mon;dw;ws];
                            temp_rem[`nif]:nf;
                            temp_rem[`month]:mon;
                            `nif`month xcols temp_rem};

# 4. Generación de las tablas

In [11]:
employees:cemployee[n_Employees];


rem1:raze ctemp_month'[employees[`contractType];
                       employees[`nif]];

rem2: rem1 lj `nif xkey select workingSchedule, cnae, nif from employees

remunerations:raze ctemp_rem'[rem2[`month];
                              rem2[`daysWorked];
                              rem2[`nif];
                              rem2[`workingSchedule];
                              rem2[`cnae]];

/---------- añadir complemento salarial ----------

tabtriennium:delete from 
                (ungroup select triennium:1 xbar("i"$distinct month-"i"$min distinct month)%36,
                                distinct month
                          by nif from remunerations)
             where triennium=0f;

tabtriennium: tabtriennium lj select salamount: sum amount by nif, month from remunerations where (max remunerationType=/:`sb`ge);

newRows: flip `nif`month`remunerationType`amount!(
               tabtriennium[`nif];
               tabtriennium[`month];
               `cs;
               0.03 * tabtriennium[`salamount] * tabtriennium[`triennium]);

remunerations,:newRows;
/---------- añadir complemento salarial ----------

Si se quisiese ver de forma más visual de las remuneraciones se puede pivotar la tabla de la siguiente manera:

In [12]:
show 0^exec `sb`cs`hefm`her`ge`se`pns`od#(remunerationType!amount) by nif,month from remunerations;

nif       month  | sb       cs       hefm     her      ge       se       pns od
-----------------| ------------------------------------------------------------
00193031S 2021.07| 619.5792 0        0        23.23422 103.2632 0        0   0 
00193031S 2021.08| 2232.243 0        0        0        372.0406 0        0   0 
00193031S 2021.09| 2236.206 0        0        0        372.7011 0        0   0 
00193031S 2021.10| 2194.139 0        0        54.85346 365.6898 0        0   0 
00193031S 2022.07| 638.4244 0        0        0        106.4041 0        0   0 
00193031S 2022.08| 2108.41  0        0        0        351.4017 0        0   0 
00193031S 2022.09| 2305.703 0        0        0        384.2838 0        0   0 
00193031S 2022.10| 2199.798 0        0        0        366.6329 0        0   0 
00193031S 2023.07| 654.0965 0        0        0        109.0161 0        0   0 
00193031S 2023.08| 2224.508 0        0        0        370.7513 0        0   0 
00193031S 2023.09| 2228.972 0        0  

# 5. Guardar datos

In [13]:
`:data/employees set employees;
`:data/remunerations set remunerations;

In [14]:
count employees
count remunerations

1000


206638
